In [1]:
 from database_handling.DataDownload import DataDownloader
# from database_handling.DataHandleAndOtherHelpers import DataHandler
 from database_handling.DataUpload import DataUploader
# from database_handling.DataDelete import DataDeleter
from database_handling.KeycloakLogin import KeycloakLogin
from scrapers.SpiegelScraper import SpiegelScraper
from text_analysis.NEExtractor import NEExtractor
from text_analysis.Summarizer import Summarizer
from text_analysis.TopicExtractor import TopicExtractor
from text_analysis.Vectorizers import Vectorizer

import transformers
import json
import gc
import torch
import logging



2024-10-01 09:19:47.527921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-01 09:19:47.632991: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 09:19:47.658289: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 09:19:48.142305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
!docker --version

/bin/bash: docker: command not found


In [3]:
# Suppress all logging below CRITICAL level
logging.basicConfig(level=logging.DEBUG)

scraper = SpiegelScraper(headless=True)

In [4]:
scraper = SpiegelScraper(headless=True)
scraper.start_browser()
scraper.login()

ERROR:scrapers.BaseScraper:Failed to start browser: Message: Process unexpectedly closed with status 255



WebDriverException: Message: Process unexpectedly closed with status 255


In [4]:
#scraper.driver.save_screenshot("screenshot.png")

In [5]:
all_found_urls = scraper.get_article_urls()

INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/
INFO:scrapers.BaseScraper:Found 170 unique article URLs on the current page: https://www.spiegel.de/
INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/fuermich/
INFO:scrapers.BaseScraper:Found 5 unique article URLs on the current page: https://www.spiegel.de/fuermich/
INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/schlagzeilen/
INFO:scrapers.BaseScraper:Found 310 unique article URLs on the current page: https://www.spiegel.de/schlagzeilen/
INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/spiegel/
INFO:scrapers.BaseScraper:Found 5 unique article URLs on the current page: https://www.spiegel.de/spiegel/print/index-2024-39.html
INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/audio/
INFO:scrapers.BaseScraper:Found 22 unique article URLs on the current page: https://www.spiegel.de/audio/
INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/plus/
INFO:scrapers.BaseScr

In [6]:
len(all_found_urls)

993

In [7]:
# get the token for the database
keycloak_login = KeycloakLogin()
token = keycloak_login.get_token()

INFO:root:Loading credentials from file: credentials_keycloak.txt
INFO:root:Getting token
INFO:root:Token is None, refreshing token
INFO:root:Refreshing token
INFO:root:Initializing KeycloakOpenID client
INFO:root:Token refreshed successfully. Token expires at: 2024-09-24 12:06:59.240133


In [8]:
data_downloader = DataDownloader(token)

In [9]:
batch_size = 30
all_urls_already_in_db = []
# Loop through all the URLs in chunks of 50
for i in range(0, len(all_found_urls), batch_size):
    # Get the current batch of URLs (from i to i + batch_size or up to the end of the list)
    current_batch = all_found_urls[i:i + batch_size]
    
    # Fetch data for the current batch
    response = data_downloader.get_content_rehydrate(url=current_batch)
    
    # Extract URLs from the 'items' in the response and store them in the list
    batch_urls = [item['url'] for item in response.get('items', [])]
    
    # Extend the combined list with the URLs from this batch
    all_urls_already_in_db.extend(batch_urls)
    print(f"Processed batch", i)
# At the end of the loop, all_urls_already_in_db contains all the URLs
print(f"Total URLs already in DB: {len(all_urls_already_in_db)}")

Processed batch 0
Processed batch 30
Processed batch 60
Processed batch 90
Processed batch 120
Processed batch 150
Processed batch 180
Processed batch 210
Processed batch 240
Processed batch 270
Processed batch 300
Processed batch 330
Processed batch 360
Processed batch 390
Processed batch 420
Processed batch 450
Processed batch 480
Processed batch 510
Processed batch 540
Processed batch 570
Processed batch 600
Processed batch 630
Processed batch 660
Processed batch 690
Processed batch 720
Processed batch 750
Processed batch 780
Processed batch 810
Processed batch 840
Processed batch 870
Processed batch 900
Processed batch 930
Processed batch 960
Processed batch 990
Total URLs already in DB: 292


In [10]:
len(all_urls_already_in_db)

292

In [11]:
keycloak_login._refresh_token()

INFO:root:Refreshing token
INFO:root:Token refreshed successfully. Token expires at: 2024-09-24 12:08:25.621406


In [12]:
data_uploader = DataUploader(token)

In [13]:
responses_for_last_online_verification_date_patch = data_uploader.patch_last_online_verification_date(all_urls_already_in_db)

In [16]:
len(responses_for_last_online_verification_date_patch)

292

In [ ]:
responses_for_last_online_verification_date_patch[0]

In [17]:
keycloak_login._refresh_token()

INFO:root:Refreshing token
INFO:root:Token refreshed successfully. Token expires at: 2024-09-24 12:09:53.346373


In [20]:
# Filter the dictionaries based on the 'url' key
articles_list_for_new_scraping = [url for url in all_found_urls if url not in all_urls_already_in_db]

In [21]:
articles = scraper.scrape(articles_list_for_new_scraping[0:5])

INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/wirtschaft/unternehmen/deutschen-boerse-geraet-ins-visier-von-eu-kartellwaechtern-a-3d667256-d500-4dc0-8c07-b56fa1c4ba8b
INFO:scrapers.BaseScraper:Extracted content from https://www.spiegel.de/wirtschaft/unternehmen/deutschen-boerse-geraet-ins-visier-von-eu-kartellwaechtern-a-3d667256-d500-4dc0-8c07-b56fa1c4ba8b
INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/sport/fussball/em-2024-toni-kroos-mit-ironischer-reaktion-auf-uefa-bericht-zu-marc-cucurellas-handspiel-a-2ff34607-54c7-432e-b565-1844946e4102
INFO:scrapers.BaseScraper:Extracted content from https://www.spiegel.de/sport/fussball/em-2024-toni-kroos-mit-ironischer-reaktion-auf-uefa-bericht-zu-marc-cucurellas-handspiel-a-2ff34607-54c7-432e-b565-1844946e4102
INFO:scrapers.BaseScraper:Navigated to https://www.spiegel.de/reise/europa/couchsurfing-in-der-ukraine-meine-reise-durch-ein-land-im-krieg-a-15963734-b1d0-49ee-9921-30f1fbd4a475
INFO:scrapers.BaseScraper:

In [1]:
def clear_gpu_memory():
    """Clears GPU memory and forces garbage collection."""
    torch.cuda.empty_cache()
    gc.collect()

def process_articles_in_batches(text_analysis_class, method_name, articles, batch_size):
    """Process articles in batches using the specified text analysis class and method."""
    processor = text_analysis_class()
    method = getattr(processor, method_name)

    for i in range(0, len(articles), batch_size):
        batch = articles[i:i + batch_size]
        batch = method(batch)

        # Reassign the processed batch back to the main list
        articles[i:i + batch_size] = batch

        clear_gpu_memory()  # Clear memory after each batch

    del processor  # Delete the processor instance to free up GPU memory
    clear_gpu_memory()

In [24]:
import torch
import gc
import json
    
# Define your batch size
batch_size = len(articles)  # Adjust this based on your GPU capacity

# Process with Entity Extractor
print("Starting entity extraction...")
process_articles_in_batches(NEExtractor, 'extract_entities', articles, batch_size)
print("Entity extraction completed.")

# Process with Topic Extractor
print("Starting topic extraction...")
process_articles_in_batches(TopicExtractor, 'extract_topics', articles, batch_size)
print("Topic extraction completed.")

# Process with Vectorizer
print("Starting vectorization...")
process_articles_in_batches(Vectorizer, 'vectorize', articles, batch_size)
print("Vectorization completed.")

# Process with Vectorizer
print("Starting summarization...")
process_articles_in_batches(Summarizer, 'summarize', articles, batch_size)
print("Summarization completed.")



INFO:text_analysis.NEExtractor:Initializing NEExtractor with model: flair/ner-german-large


Starting entity extraction...
2024-09-24 12:08:22,267 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>


INFO:text_analysis.NEExtractor:Tagger instantiated successfully
INFO:text_analysis.NEExtractor:Starting entity extraction for 5 articles
INFO:text_analysis.NEExtractor:Applying entity extraction to all articles
Map: 100%|██████████| 5/5 [00:02<00:00,  2.32 examples/s]
INFO:text_analysis.NEExtractor:Inferring geographical focus
INFO:text_analysis.NEExtractor:Starting geographical focus inference for 5 articles


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[1]]
[1] "newsmap"   "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
[1] "quanteda"  "newsmap"   "stats"     "graphics"  "grDevices" "utils"    
[7] "datasets"  "methods"   "base"     

[[3]]
 [1] "dplyr"     "quanteda"  "newsmap"   "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     

[[4]]
 [1] "countrycode" "dplyr"       "quanteda"    "newsmap"     "stats"      
 [6] "graphics"    "grDevices"   "utils"       "datasets"    "methods"    
[11] "base"       

[[5]]
 [1] "jsonlite"    "countrycode" "dplyr"       "quanteda"    "newsmap"    
 [6] "stats"       "graphics"    "grDevices"   "utils"       "datasets"   
[11] "methods"     "base"       



Package version: 4.1.0
Unicode version: 13.0
ICU version: 66.1
Parallel computing: disabled
See https://quanteda.io for tutorials and examples.

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

INFO:text_analysis.NEExtractor:Geographical focus inference completed successfully
INFO:text_analysis.NEExtractor:Entity extraction and geographical focus inference complete


...

Topic extraction completed.
Starting vectorization...


/home/jovyan/work/Patrick/m3-scraper/patrick_myenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: T-Systems-onsite/german-roberta-sentence-transformer-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: deutsche-telekom/gbert-large-paraphrase-euclidean
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained Se

Vectorization completed.
Starting summarization...


Map: 100%|██████████| 5/5 [00:27<00:00,  5.44s/ examples]


Summarization completed.


In [25]:
# Remove main_text and lead_text from articles to save space before uploading
for article in articles:
    article.pop('main_text', None)
    article.pop('lead_text', None)


In [26]:
# Upload each article to the database
print("Beginning article upload...")
responses = []
keycloak_login = KeycloakLogin()
token = keycloak_login.get_token()
data_uploader = DataUploader(token)

INFO:root:Loading credentials from file: credentials_keycloak.txt
INFO:root:Getting token
INFO:root:Token is None, refreshing token
INFO:root:Refreshing token
INFO:root:Initializing KeycloakOpenID client
INFO:root:Token refreshed successfully. Token expires at: 2024-09-24 12:19:18.127733


Beginning article upload...


In [27]:
for article in articles:
    response = data_uploader.post_content(article)
    responses.append(response)

In [28]:
# Save the responses to a JSON file
with open('responses.json', 'w') as f:
    json.dump(responses, f)

print("Article upload completed.")

Article upload completed.
